In [57]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import numpy as np
import random
import pandas

In [58]:
def generate_dataset():
    col_names = ['ts','user','x_coordinate','y_coordinate','color', 'label']
    data = pandas.read_csv('four_blocks.txt', names = col_names, header=None, skiprows=1)
#     print(type(data))
    
#     ts = data.ts.values
    
#     print(type(ts))
    return data

In [59]:
data = generate_dataset()
dim = data.ts.values.shape[0]
num_labels = 4
print(dim)
#type(data.x_coordinate.values[1])
train_x = np.concatenate((data.ts.values.reshape(dim,1),
                          data.user.values.reshape(dim,1),
                          data.x_coordinate.values.reshape(dim,1),
                          data.y_coordinate.values.reshape(dim,1),
                          data.color.values.reshape(dim,1)), axis=1)
print(train_x.shape)


10000
(10000, 5)


In [61]:
train_y = np.zeros((dim,num_labels))
for i in range(dim):
    train_y[i][data.label.values[i]] = 1
print(train_y.shape)

(10000, 4)


In [62]:
def build_model():
    x = tf.placeholder(tf.float32, shape=(None, 5), name='x')    
    y = tf.placeholder(tf.float32, shape=(None, num_labels), name='y') 
    
    W1 = tf.Variable(tf.random_normal([5, 128], stddev=0.03), name='W1')
    b1 = tf.Variable(tf.random_normal([128]), name='b1')
    
    W2 = tf.Variable(tf.random_normal([128, num_labels], stddev=0.03), name='W2')
    b2 = tf.Variable(tf.random_normal([num_labels]), name='b2')

    
    hidden_out = tf.add(tf.matmul(x, W1), b1)
    hidden_out = tf.nn.relu(hidden_out)
    logits = tf.add(tf.matmul(hidden_out, W2), b2)
    prediction = tf.nn.softmax(logits)
    y_clipped = tf.clip_by_value(prediction, 1e-10, 0.9999999)
    
    loss = -tf.reduce_mean(tf.reduce_sum(y * tf.log(y_clipped) + (1 - y) * tf.log(1 - y_clipped), axis=1))
    
    
    train_op = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(loss)
    init = tf.global_variables_initializer()

    correct_prediction = tf.equal(tf.argmax(prediction, 1, name="Argmax_Pred"), tf.argmax(y, 1, name="Y_Pred"), name="Correct_Pred")
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32, name="Cast_Corr_Pred"), name="Accuracy")
    
    return x, y, init, train_op, loss, accuracy

In [63]:
def run(x, y, init, train_op, loss, accuracy):
    #define session
    with tf.Session() as sess:
        sess.run(init)
        feed_dict = {x: train_x, y: train_y}

        for i in range(100):
            print('EPOCH', i)
            _, loss_val = sess.run([train_op, loss], feed_dict)
            print('DONE WITH EPOCH')
            print("loss:", loss_val)

        print("Accuracy:", sess.run( accuracy, feed_dict))

In [88]:
x, y, init, train_op, loss, accuracy = build_model()
run(x, y, init, train_op, loss, accuracy)

EPOCH 0
DONE WITH EPOCH
loss: 2.7291324
EPOCH 1
DONE WITH EPOCH
loss: 2.3373563
EPOCH 2
DONE WITH EPOCH
loss: 2.6370625
EPOCH 3
DONE WITH EPOCH
loss: 2.155892
EPOCH 4
DONE WITH EPOCH
loss: 1.5095977
EPOCH 5
DONE WITH EPOCH
loss: 1.4452391
EPOCH 6
DONE WITH EPOCH
loss: 1.4261378
EPOCH 7
DONE WITH EPOCH
loss: 1.471675
EPOCH 8
DONE WITH EPOCH
loss: 1.3207442
EPOCH 9
DONE WITH EPOCH
loss: 1.2983655
EPOCH 10
DONE WITH EPOCH
loss: 1.2054574
EPOCH 11
DONE WITH EPOCH
loss: 1.1939104
EPOCH 12
DONE WITH EPOCH
loss: 1.1360381
EPOCH 13
DONE WITH EPOCH
loss: 1.1260457
EPOCH 14
DONE WITH EPOCH
loss: 1.0769045
EPOCH 15
DONE WITH EPOCH
loss: 1.0678757
EPOCH 16
DONE WITH EPOCH
loss: 1.0309613
EPOCH 17
DONE WITH EPOCH
loss: 1.0206023
EPOCH 18
DONE WITH EPOCH
loss: 0.98523283
EPOCH 19
DONE WITH EPOCH
loss: 0.9785542
EPOCH 20
DONE WITH EPOCH
loss: 0.9545221
EPOCH 21
DONE WITH EPOCH
loss: 0.94873625
EPOCH 22
DONE WITH EPOCH
loss: 0.9294432
EPOCH 23
DONE WITH EPOCH
loss: 0.9232857
EPOCH 24
DONE WITH EPOCH
l

In [193]:

x = tf.placeholder(tf.float32, shape=(None, 5), name='x')    
y = tf.placeholder(tf.int64, shape=(None, num_labels), name='y') 

layer1 = tf.contrib.layers.fully_connected(x, 128, tf.nn.relu)
predictions = tf.contrib.layers.fully_connected(layer1, num_labels, tf.nn.softmax)
loss = tf.losses.softmax_cross_entropy(onehot_labels = y, logits = predictions)
correct_prediction = tf.equal(tf.argmax(predictions, 1, name="Argmax_Pred"), tf.argmax(y, 1, name="Y_Pred"), name="Correct_Pred")
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32, name="Cast_Corr_Pred"), name="Accuracy")

train_op = tf.train.GradientDescentOptimizer(learning_rate = 0.01).minimize(loss)


#define session
sess = tf.Session()
sess.run(tf.global_variables_initializer())
feed_dict = {x: train_x, y: train_y}


for i in range(100):
    print('EPOCH', i)
    _, loss_val = sess.run([train_op, loss], feed_dict)
    print('DONE WITH EPOCH')
    print("loss:", loss_val)

print("Accuracy:", sess.run( accuracy, feed_dict = {x: train_x, y: train_y}))
    
    

EPOCH 0
DONE WITH EPOCH
loss: 1.5720711
EPOCH 1
DONE WITH EPOCH
loss: 1.5418348
EPOCH 2
DONE WITH EPOCH
loss: 1.4929717
EPOCH 3
DONE WITH EPOCH
loss: 1.468345
EPOCH 4
DONE WITH EPOCH
loss: 1.4578428
EPOCH 5
DONE WITH EPOCH
loss: 1.4527224
EPOCH 6
DONE WITH EPOCH
loss: 1.4474506
EPOCH 7
DONE WITH EPOCH
loss: 1.4421428
EPOCH 8
DONE WITH EPOCH
loss: 1.4371489
EPOCH 9
DONE WITH EPOCH
loss: 1.4350188
EPOCH 10
DONE WITH EPOCH
loss: 1.4493898
EPOCH 11
DONE WITH EPOCH
loss: 1.5194391
EPOCH 12
DONE WITH EPOCH
loss: 1.4289603
EPOCH 13
DONE WITH EPOCH
loss: 1.446091
EPOCH 14
DONE WITH EPOCH
loss: 1.5298197
EPOCH 15
DONE WITH EPOCH
loss: 1.4362342
EPOCH 16
DONE WITH EPOCH
loss: 1.4927328
EPOCH 17
DONE WITH EPOCH
loss: 1.5227529
EPOCH 18
DONE WITH EPOCH
loss: 1.422243
EPOCH 19
DONE WITH EPOCH
loss: 1.4389552
EPOCH 20
DONE WITH EPOCH
loss: 1.5361227
EPOCH 21
DONE WITH EPOCH
loss: 1.4411899
EPOCH 22
DONE WITH EPOCH
loss: 1.5159793
EPOCH 23
DONE WITH EPOCH
loss: 1.4766953
EPOCH 24
DONE WITH EPOCH
loss